## Что такое Cython?

Cython – это:
* язык программирования, который сочетает python со статической типизацией C/C++;
* компилятор, который транслирует код, написанный на Cython в код на C/C++.

Помимо этого, с помощью Cython можно оборачивать код на C/C++ и вызывать его из python.

**ВАЖНО:** Не путайте Cython и Cpython (это интерпретатор языка python).

In [ ]:
%load_ext cython

## Мотивирующий пример

Давайте попробуем найти сумму элементов в двумерном массиве с помощью python.

In [ ]:
import numpy as np

In [ ]:
def sum_array(a):
    n_rows, n_cols = a.shape

    s = 0
    for i in range(n_rows):
        for j in range(n_cols):
            s += a[i, j]
    return s

In [ ]:
a = np.random.random((300, 300))

In [ ]:
%%timeit

_ = sum_array(a)

In [ ]:
%%timeit

_ = a.sum()

In [ ]:
%%cython

cimport numpy as np

cpdef float sum_array(np.ndarray[np.float_t, ndim=2] a):
    cdef int n_rows = a.shape[0], n_cols = a.shape[1]
    cdef int i = 0, j = 0
    cdef float s = 0

    for i in range(n_rows):
        for j in range(n_cols):
            s += a[i, j]
    return s

In [ ]:
%%timeit

_ = sum_array(a)

### Заглянем немного внутрь

In [ ]:
def fib_py(n):
    a, b = 0, 1
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%%timeit

fib_py(50)

Добавим немного типизации в нашу программу. Для создания статически типиизированной переменной используется ключевое слово `cdef`.

In [ ]:
%%cython

def fib_cy(int n):
    cdef int i
    cdef double a = 0.0, b = 1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [ ]:
%%timeit

fib_cy(50)

В Cython есть 3 вида функциий:
* для объявления, которых используется `cdef` – С-функция, написанная на Cython (нельзя вызвать из python);
* для объявления, которых используется `def` – функция, написанная на Cython (можно вызвать из python);
* для объявления, которых используется `cpdef` – функция, совмещающая возможности `cdef` и `def`.

In [ ]:
%%cython

def fact_typed(long n):
    if n <= 1:
        return 1
    return n * fact_typed(n - 1)

In [ ]:
fact_typed(10)

In [ ]:
%%cython

cdef long fact_c(long n):
    if n <= 1:
        return 1
    return n * fact_c(n - 1)

def fact_wrapper(long n):
    return fact_c(n)

In [ ]:
fact_c(10)

In [ ]:
fact_wrapper(10)

In [ ]:
%%cython

cpdef long fact_ctyped(long n):
    if n <= 1:
        return 1
    return n * fact_ctyped(n - 1)

In [ ]:
fact_ctyped(10)

В Cython можно создавать свои структуры.

In [ ]:
%%cython

cdef struct complex_c:
    float real
    float imag
    
cdef complex_c a = complex_c(3.1415, -1.0)

### Cython и NumPy

Рассмотрим простенький пример – умножение двух матриц.

In [ ]:
X = np.random.random(size=(150, 100))
Y = np.random.random(size=(250, 100))

X.shape, Y.shape

In [ ]:
def py_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    Z = np.zeros((n_xrows, n_ycols))

    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]

    return Z

In [ ]:
%%timeit

Z = py_matrix_product(X, Y)

Для профилирования нашей программы добавим флаг `-a`.

In [ ]:
%%cython -a

import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    Z = np.zeros((n_xrows, n_ycols))

    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]

    return Z

In [ ]:
%%timeit

Z = cy_matrix_product(X, Y)

Добавим типы.

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np

def cy_matrix_product(X, Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols))

    cdef int i = 0, k = 0, j = 0

    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]

    return Z

In [ ]:
%%timeit

Z = cy_matrix_product(X, Y)

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np

def cy_matrix_product(np.ndarray[np.float64_t, ndim=2] X,
                      np.ndarray[np.float64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    
    cdef np.ndarray[np.float64_t, ndim=2] Z
    Z = np.zeros((n_xrows, n_ycols))

    cdef int i = 0, k = 0, j = 0

    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]

    return Z

In [ ]:
%%timeit

Z = cy_matrix_product(X, Y)

Отключаем проверки в Cython.

In [ ]:
%%cython -a

import numpy as np
cimport numpy as np

cimport cython

@cython.boundscheck(False)    # выход за границы массива
@cython.overflowcheck(False)  # проверка на переполнение
@cython.wraparound(False)     # отрицательная индексация у массивов
def cy_matrix_product(np.ndarray[np.float64_t, ndim=2] X,
                      np.ndarray[np.float64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    
    cdef np.ndarray[np.float64_t, ndim=2] Z
    Z = np.zeros((n_xrows, n_ycols))

    cdef int i = 0, k = 0, j = 0

    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]

    return Z

In [ ]:
%%timeit

Z = cy_matrix_product(X, Y)